In [1]:
import AD2_w_utils as ad
# from AD2_w_utils import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import scipy
import scanpy as sc
import sklearn
import copy
import importlib
# %matplotlib inline


In [19]:
import AD2_w_utils as ad
importlib.reload(ad)


<module 'AD2_w_utils' from '/Users/evanli/Documents/EvanPys/Progress/AD2_w_utils.py'>

## Read PBMC dataset

In [ ]:
data_path = '/Users/evanli/Documents/Research_datasets/PBMC_Hao/'
# os.chdir('/home/evanlee/PBMC_Hao/')
# data_path = ''
adata_raw = sc.read(data_path + 'Hao_PBMC.h5ad')
print(adata_raw.shape)  # row is cells, column is gene
# (161764, 20568)

# standardize expression data
adata_std = copy.deepcopy(adata_raw)
# Total-count normalize the data matrix X to 10,000 reads per cell
sc.pp.normalize_total(adata_std, target_sum=1e4)
# Log
sc.pp.log1p(adata_std)


In [ ]:
# Subset 10 percent
import random

random.seed(0)
rand_idx = random.sample(range(adata_raw.shape[0]), int(adata_raw.shape[0]*0.1))
subset = adata_raw[rand_idx, :]
subset_std = adata_std[rand_idx, :]


In [ ]:
subset.write('subset_10percent.h5ad')
subset_std.write('subset_10percent_std.h5ad')

In [ ]:
# celltype count
subset.obs['celltype.l2'].value_counts()

## Read PBMC subset

In [2]:
data_path = '/Users/evanli/Documents/Research_datasets/PBMC_Hao/'
# os.chdir('/home/evanlee/PBMC_Hao/')
# data_path = ''
subset = sc.read(data_path + 'subset_10percent.h5ad')
print(subset.shape)  # row is cells, column is gene
# (161764, 20568)

# standardized susbet
subset_std = sc.read(data_path + 'subset_10percent_std.h5ad')
print(subset_std.shape)  # row is cells, column is gene


(16176, 20568)
(16176, 20568)


## Lambda tuning for demo dataset

In [6]:
# Demo
url_1 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/ASV_vst.txt'
url_2 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/ASV_table.txt'
url_3 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/metadata.txt'
Data = pd.read_csv(url_1, sep="\t")
Data = Data.T           # Variance-stabilizing transformation was conducted by DESeq2
# we using z-normalization data as input-data
Data_std = scipy.stats.zscore(Data, axis=0, ddof=0)
RawData = pd.read_csv(url_2, sep="\t")
RawData = RawData.T  # Raw count data, was used as an assessment of prevalence
Cohort = pd.read_csv(url_3, sep="\t")                        # Metadata
Label = Cohort['Class'].tolist()

print('This data contains',
      Data_std.shape[0], 'samples and', Data_std.shape[1], 'features')
print(Label[0:10], np.unique(Label))

This data contains 129 samples and 6661 features
['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal'] ['Cancer' 'Normal']


In [ ]:
# Lambda tuning
# auto_scale(X_input, X_raw, y, step=50)
log_lmbd_range = ad.auto_scale(Data_std, RawData, Label, step=50)

lmbd_range = np.exp(log_lmbd_range)
print(lmbd_range)


In [16]:
import AD2_0728 as ad728

llr = ad728.auto_scale(Data_std, RawData, Label, step=50)
demo_range = np.exp(llr)
print(demo_range)

[1.00000000e-08 1.38949549e-08 1.93069773e-08 2.68269580e-08
 3.72759372e-08 5.17947468e-08 7.19685673e-08 1.00000000e-07
 1.38949549e-07 1.93069773e-07 2.68269580e-07 3.72759372e-07
 5.17947468e-07 7.19685673e-07 1.00000000e-06 1.38949549e-06
 1.93069773e-06 2.68269580e-06 3.72759372e-06 5.17947468e-06
 7.19685673e-06 1.00000000e-05 1.38949549e-05 1.93069773e-05
 2.68269580e-05 3.72759372e-05 5.17947468e-05 7.19685673e-05
 1.00000000e-04 1.38949549e-04 1.93069773e-04 2.68269580e-04
 3.72759372e-04 5.17947468e-04 7.19685673e-04 1.00000000e-03
 1.38949549e-03 1.93069773e-03 2.68269580e-03 3.72759372e-03
 5.17947468e-03 7.19685673e-03 1.00000000e-02 1.38949549e-02
 1.93069773e-02 2.68269580e-02 3.72759372e-02 5.17947468e-02
 7.19685673e-02 1.00000000e-01]


## Lambda tuning for CD4 CTL

In [4]:
outdir = './tuning'
os.path.exists(outdir)

False

In [5]:
# Binary classification of CD4 CTL
labels = [1 if x == 'CD4 CTL' else 0 for x in subset.obs['celltype.l2']]

# create index for CD4 CTL
# CD4_CTL_indices = [idx for idx, cell_type in enumerate(
#     subset.obs['celltype.l2']) if cell_type == 'CD4 CTL']
CD4_CTL_indices = [idx for idx, label in enumerate(labels) if label == 1]


In [9]:
# binary_cross_entropy error
log_lmbd_range = ad.auto_scale(subset_std.X, subset.X, labels)
lmbd_range = np.exp(log_lmbd_range)
print(lmbd_range)

Getting prevalance
isspmatrix_csr prevalence
Lambda: 1e-10
iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
iter: 14
iter: 15
iter: 16
iter: 17
iter: 18
iter: 19
iter: 20
iter: 21
iter: 22
iter: 23
iter: 24
iter: 25
iter: 26
iter: 27
iter: 28
iter: 29
iter: 30
iter: 31
iter: 32
iter: 33
iter: 34
iter: 35
iter: 36
iter: 37
iter: 38
iter: 39
Hessian initialization fail, gradient diff = 0
Lambda: 1e-09
iter: 0
iter: 1
iter: 2


RuntimeError: all elements of input should be between 0 and 1

In [ ]:
result_dict = lambda_tuning_evan(subset_std.X, subset.X, labels, lmbd_range, outdir)

In [20]:
result_dict = ad.lambda_tuning_evan(subset_std.X, subset.X, labels, demo_range, outdir)

Getting prevalance
isspmatrix_csr prevalence


TypeError: bad operand type for unary ~: 'list'

In [16]:
type(subset.X)



scipy.sparse.csr.csr_matrix

In [13]:
exam_range = [1/(10**i) for i in np.arange(10,-1,-1)]
exam_range

[1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0]